In [1]:
import sys
sys.path.insert(0, '../odbc/')
import query as qry

In [2]:
df_vti = qry.load_frame('s_sih_2019')
df_ooa = qry.load_frame('s_ooa_2019')
df_dos = qry.load_frame('s_ooa_dos')
df_opl = qry.load_frame('r_opl')

In [ ]:
df_dos = df_dos.loc[df_dos['PROCESSTAP'].str.contains('O_DATUM')]
df_dos = df_dos.loc[df_dos['GESLOTEN_ANTWOORD_CODE'] == 'APRIL']
df_dos = df_dos.loc[df_dos['FACULTEIT'] == 'BETA']

In [ ]:
cols = ['STUDENTNUMMER', 'OPLEIDING', 'CROHO']
df_ooa_c = df_ooa.merge(df_vti[cols], on=['STUDENTNUMMER', 'OPLEIDING'])

In [ ]:
cols = ['IO_AANVR_ID', 'GESLOTEN_ANTWOORD_CODE']
df_m = df_dos[cols].merge(df_ooa_c, on=['IO_AANVR_ID'])

In [6]:
df_m['OPLEIDING'].value_counts()

BIOL    206
INCA    104
SCHK     71
INKU     62
NAST     58
WISK     52
WSKT     14
Name: OPLEIDING, dtype: int64

In [7]:
cols = ['STUDENTNUMMER', 'CROHO', 'OPLEIDING']
df_inlees = df_m[cols].copy()

In [8]:
nast = df_inlees.loc[df_inlees['OPLEIDING'] == 'NAST']['STUDENTNUMMER']
wisk = df_inlees.loc[df_inlees['OPLEIDING'] == 'WISK']['STUDENTNUMMER']
schk = df_inlees.loc[df_inlees['OPLEIDING'] == 'SCHK']['STUDENTNUMMER']
inca = df_inlees.loc[df_inlees['OPLEIDING'] == 'INCA']['STUDENTNUMMER']
inku = df_inlees.loc[df_inlees['OPLEIDING'] == 'INKU']['STUDENTNUMMER']

In [9]:
stud_nast = df_inlees['STUDENTNUMMER'].isin(nast)
stud_wisk = df_inlees['STUDENTNUMMER'].isin(wisk)
stud_schk = df_inlees['STUDENTNUMMER'].isin(schk)
stud_inca = df_inlees['STUDENTNUMMER'].isin(inca)
stud_inku = df_inlees['STUDENTNUMMER'].isin(inku)

In [10]:
# INKU/INCA
filt = stud_inca & stud_inku
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'INKU'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INKU'

# WISK/INCA
filt = stud_wisk & stud_inca
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

# NAST/SCHK
filt = stud_nast & stud_schk
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'SCHK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'SCHK'

# NAST/WIS
filt = stud_nast & stud_wisk
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['OPLEIDING'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

In [17]:
df_inlees.query("VOOR_ACTIVITEIT_BETA_TWIN == VOOR_ACTIVITEIT_BETA_TWIN").to_excel('output/inlees_bestand_beta.xlsx')